## Part 1

:::{.rmenv title="Exercise (Installation)"}
Follow the [quick start](quickstart) instructions to install Julia, a code editor, and the git version control system.
:::

:::{.solenv}
If you're stuck on setup, please ask an instructor or TA for help!
:::

:::{.rmenv title="Exercise (Hello World)"}
Get Julia to print "Hello World".
:::

:::{.solenv}
Output text with the `println` function.


In [ ]:
println("Hello World!")

String concatentation in Julia uses the `*` symbol.


In [ ]:
println("Hello" * " " * "World!")

Print everything in a vector by using broadcasting. To broadcast, append `.` to your function, and pass it something you can iterate over (like a vector). Or, use `map` or `foreach`.


In [ ]:
# Both of these return a vector of `nothing`s:

# println.(["Hello", "World!"])
# map(println, ["Hello", "World!"])

# This one produces side effects only:
foreach(println, ["Hello", "World!"])

Perform string interpolation using `$`.


In [ ]:
favorite_library = "AlgebraicDynamics.jl"

println("My favorite AlgebraicJulia package is $(favorite_library)!")

:::

:::{.rmenv title="Exercise (Package management)"}
Install a Julia package, such as `Catlab` or `AlgebraicDynamics`, using the [Julia package manager](https://docs.julialang.org/en/v1/stdlib/Pkg/).
:::

:::{.solenv}
From a Julia REPL, enter the package manager by typing `]`. Create a new project, which will organize the packages you're using for this workshop, by typing
`pkg> activate .`
Next, add Catlab to your project by typing
`pkg> add Catlab`
You should see a bunch of stuff happening, including precompilation of the Catlab code.

When that's finished, hit backspace to get out of the package manager and type
```julia
using Catlab
```
to load Catlab into your REPL session. To make sure it worked right, try something like
```julia
g = path_graph(Graph, 7)
```
to produce your first graph in Catlab!
:::


:::{.rmenv title="Exercise (Sharing is caring)"}
Fork and clone the [ACT2023Tutorial](https://github.com/AlgebraicJulia/ACT2023Tutorial) repository, make a change, commit it, and create a pull request.
:::

:::{.solenv}
1. Log into [github.com](https://github.com)
2. Navigate to [github.com/AlgebraicJulia/ACT2023Tutorials](https://github.com/AlgebraicJulia/ACT2023Tutorials)
3. By the top right of the page there click the "Fork" button to the left of the "Star" button.
4. On the next page click "Create fork", this will create a copy of the repository on your own GitHub account
5. Clone your fork (replacing `<user_name>` with your own GitHub username)
   ```sh
   git clone https://github.com/<user_name>/ACT2023Tutorials.git
   ```
6. Navigate into the solutions folder and create a new directory with your name (replacing `<your_name>` with your own name)
   ```sh
   cd ACT2023Tutorials/solutions
   mkdir <your_name>
   ```
7. Put some code in there!
   ```sh
   echo "print('Hello, World!')" > hello_world.jl
   ```
8. Stage, commit, and push your changes
   ```sh
   git add hello_world.jl
   git commit -m "Adding solutions for <your_name>"
   git push
   ```
9. Navigate to your fork on GitHub (replacing `<user_name>` with your GitHub username): `https://github.com/<user_name>/ACT2023Tutorials`
10. Click the "Pull requests" tab across the top of the page and click the "New pull request". This will automatically create a new pull request on the original repository from your fork
11. Click "Create pull request"
12. Put a short title and description and click "Create pull request"
:::

:::{.rmenv title="Exercise (Finite sets)"}
Write a function that computes the product of two finite sets, along with the projection maps out.
:::

:::{.solenv}
For convenience, we'll use the types `FinSet` and `FinFunction` exported by Catlab, which are similar to but more general than those in the problem statement.


In [ ]:
#| output: false
using Catlab

function product_projections(A::FinSet{Int}, B::FinSet{Int})
  m, n = length(A), length(B)
  AB = FinSet(m*n)
  
  # In this calculation, `÷` (short for `div` function) is truncating integer
  # division and `%` (short for `mod` function) is the modulus.
  # Notice that we're using the skeleton of FinSet with sets `1:n` but the
  # calculation below is most natural for sets `0:(n-1)`.
  πA = FinFunction(i -> (i-1) ÷ n + 1, AB, A)
  πB = FinFunction(i -> (i-1) % n + 1, AB, B)
  
  (πA, πB)
end

An example:


In [ ]:
using Test

π₁, π₂ = product_projections(FinSet(3), FinSet(4))
@test dom(π₁) == dom(π₂) == FinSet(12)
@test collect(π₁) == [1,1,1,1, 2,2,2,2, 3,3,3,3]
@test collect(π₂) == [1,2,3,4, 1,2,3,4, 1,2,3,4]

:::

:::{.rmenv title="Exercise (Graphs)"}
Implement a graph data structure in Catlab either (a) using Catlab's support for C-sets (`@acset_type`) or (b) more directly, using finite sets (`FinSet`) and functions (`FinFunction`).
:::

:::{.solenv}

a. Graphs as C-sets:


In [ ]:
using Catlab

@present SchGph(FreeSchema) begin
   V::Ob
   E::Ob
   src::Hom(E,V)
   tgt::Hom(E,V)
end

# Index the `src` and `tgt` functions for fast lookups of inverse images
# using the `incident` function.
@acset_type Gph(SchGph, index=[:src,:tgt])

# An example graph
g = @acset Gph begin
  V = 3
  E = 2
  src = [1,1]
  tgt = [2,1]
end

   We use the name `Gph` to avoid a name conflict with Catlab's exported type
   `Graph`, but `Graph` is defined in Catlab in exactly the same way!
   
   The exported `Graph` type includes support for visualization using
   [Graphviz](https://graphviz.org/), which we can use to see what the above
   graph looks like:


In [ ]:
g = @acset Graph begin
  V = 3
  E = 2
  src = [1,1]
  tgt = [2,1]
end

to_graphviz(g, node_labels=true)

b. Using finite sets and functions:


In [ ]:
#| output: false
# Catlab exports these generic functions. We import them explicitly so that
# we can add new methods to them.
import Catlab: src, tgt, vertices, edges

struct MyGraph
  # We don't need fields for `E` and `V` since these are the domain and
  # codomain of `src` and `tgt`.
  src::FinFunction
  tgt::FinFunction

  function MyGraph(src::FinFunction, tgt::FinFunction)
    dom(src) == dom(tgt) || error("Source and target must have same domain")
    codom(src) == codom(tgt) || error("Source and target must have same codomain")
    new(src, tgt)
  end
end

# A few accessor functions. Our users shouldn't need to know the internal
# layout of the fields in the struct!
src(g::MyGraph) = g.src
tgt(g::MyGraph) = g.tgt
vertices(g::MyGraph) = codom(src(g)) # == codom(tgt(g))
edges(g::MyGraph) = dom(src(g)) # == dom(tgt(g))

   Now let's test that it works!


In [ ]:
using Test

s, t = FinFunction([1,1], 3), FinFunction([2,1], 3)
g = MyGraph(s, t)

@test vertices(g) == FinSet(3)
@test edges(g) == FinSet(2)

:::

:::{.rmenv title="Exercise (Optimizing Euler)"}
Write an optimized implementation of Euler's method that computes the derivative in place and only makes allocations when initializing the algorithm.
:::

:::{.solenv}
For the sake of comparison, let's first measure the allocations of the naive implementation of Euler's method.


In [ ]:
function euler(du::Function, init::Vector{Float64}, dt::Float64, m::Int)
  n = length(init)
  x = zeros(Float64, (m+1, n))
  x[1,:] = init
  for i in 1:m
    x[i+1,:] = x[i,:] + dt * du(x[i,:])
  end
  x
end

du(u) = [u[3], u[4], 0, -9.8]
@time trajectory = euler(du, [0, 0, 5.0, 10.0], 0.1, 1000);

In the optimized version, we pass a function `du!` that computes the derivative in-place.


In [ ]:
function inplace_euler(du!::Function, init::Vector{Float64}, dt::Float64, m::Int)
  n = length(init)
  x = zeros(Float64, (m+1, n))
  du = zeros(Float64, n)
  x[1,:] = init
  for i in 1:m
    du!(du, j -> x[i,j])
    for j in 1:n
      x[i+1,j] = x[i,j] + dt * du[j]
    end
  end
  x
end

@inline function du!(du, u)
  du[1] = u(3)
  du[2] = u(4)
  du[3] = 0
  du[4] = -9.8
end
@time trajectory′ = inplace_euler(du!, [0, 0, 5.0, 10.0], 0.1, 1000);

Notice that the number of allocations is constant in the length of the trajectory. We better also check that we get the same answer as before.


In [ ]:
@test trajectory′ ≈ trajectory

:::

# Part 2

:::{.rmenv title="Exercise (Growing rabbits)"}
Implement the dynamical system

$$ \dot{R} = \alpha R $$

using `AlgebraicDynamics`.
:::

:::{.solenv}
Define the dynamical system as a resource sharer:


In [ ]:
using AlgebraicDynamics

ṙ(r,p,t) = p.α * r
rabbit_growth = ContinuousResourceSharer{Float64}(1, ṙ)

Simulate and plot:


In [ ]:
using LabelledArrays
using DifferentialEquations, Plots

u0 = [1.0]
params = LVector(α=0.3)
tspan = (0.0, 10.0)

prob = ODEProblem(rabbit_growth, u0, tspan, params)
sol = solve(prob, Tsit5())

plot(sol, title="Exponential growth of rabbit population",
     legend=false, xlabel="time", ylabel="population size")

:::

:::{.rmenv title="Exercise (UWDs as data structure)"}
a. Write a call to the `@relation` macro that constructs the shown UWD.
b. Construct the UWD manually using the API for UWDs or the lower-level API for acsets.
:::

:::{.solenv}
Using the `@relation` macro:


In [ ]:
uwd = @relation (x,y,z) where (w,x,y,z) begin
  R(x,w)
  S(y,w)
  T(z,w)
end

to_graphviz(uwd, box_labels=:name, junction_labels=:variable)

Using the UWD API:


In [ ]:
d = RelationDiagram(3) # Create diagram with 3 outer ports
add_junctions!(d, 4, variable=[:w,:x,:y,:z]) # Add four junctions
set_junction!(d, 1:3, 2:4, outer=true) # Set junctions of outer ports

for name in [:R, :S, :T]
  add_box!(d, 2, name=name) # Add box with two ports
end
for (i, box) in enumerate(boxes(d))
  set_junction!(d, (box, 1), i+1)
  set_junction!(d, (box, 2), 1)
end

@test d == uwd

The advantange of using the programmatic API is that this code is easily generalized to a function that creates a "star-shaped UWD" of any size. You might try implementing this function as a further exercise.
:::

:::{.rmenv title="Exercise (Pendulum)"}
Implement a dynamical system modeling the trajectory of a pendulum.
:::

:::{.solenv}
We'll use the differential equation for the motion of a [simple pendulum](https://en.wikipedia.org/wiki/Pendulum_(mechanics))

$$ \frac{d^2 \theta}{dt^2} + \frac{g}{\ell} \sin\theta = 0, $$

where $g$ is the acceleration due to gravity and $\ell$ is the length of the rod. It's fun that since we're working numerically, we don't need to bother with the small-angle approximation $\sin(\theta) \approx \theta$ that you probably remember from physics class.

We turn this second-order ODE into a first-order system of ODEs by introducing a new variable, the angular velocity $\omega$:

\begin{align*}
  \dot\theta &= \omega \\
  \dot\omega &= -\frac{g}{\ell} \sin\theta
\end{align*}

Now we can plug this into AlgebraicDynamics:


In [ ]:
du(u,p,t) = [u[2], -p.g/p.ℓ * sin(u[1])]

pendulum = ContinuousResourceSharer{Float64}(2, du)

u0 = [π/4, 0]
params = LVector(g=9.8, ℓ=1)
tspan = (0.0, 10.0)

prob = ODEProblem(pendulum, u0, tspan, params)
sol = solve(prob, Tsit5())
plot(sol, title="Simple pendulum", xlabel="time", label=["θ" "ω"])

:::

:::{.rmenv title="Exercise (Graph traversal)"}
Implement a graph traversal algorithm, such as depth-first search or
breadth-first search, using the acsets API (`subpart`, `incident`).
:::

:::{.solenv}
A stack-based depth-first search algorithm:


In [ ]:
#| output: false
using Catlab
using DataStructures: Stack

""" Depth-first search in graph `g` starting at vertex `v`

The function `f` is called at every vertex in the search path.
"""
function dfs(f::Function, g::ACSet, v::Int)
  seen = Set{Int}()
  next = Stack{Int}()
  push!(next, v)
  while !isempty(next)
    v = pop!(next)
    v ∈ seen && continue
    f(v)
    push!(seen, v)
    # Add all outgoing edges to the stack.
    for e in incident(g, v, :src)
      # Syntactic sugar: `g[e, :tgt] == subpart(g, e, :tgt)``
      push!(next, g[e, :tgt])
    end
  end
end

Traversal of a directed cycle, starting at the third vertex, yields the vertices
in the expected order.


In [ ]:
g = cycle_graph(Graph, 6)
dfs(println, g, 3)

:::{.rmenv title="Exercise (Union-find)"}
Use the union-find data structure to find the connected components of a graph, by making `src(e)` and `tgt(e)` be in the same equivalence class for each edge `e`.
:::

:::{.solenv}

In [ ]:
#| output: false
using Catlab
using DataStructures: IntDisjointSets, union!, find_root

""" Find connected components of a graph.

Returns a dictionary from the component roots to lists of vertices in each
component.
"""
function connected_components(g::HasGraph)
  sets = IntDisjointSets(nparts(g, :V))
  for e in parts(g, :E)
    union!(sets, g[e,:src], g[e,:tgt])
  end

  components = Dict{Int,Vector{Int}}()
  for v in parts(g, :V)
    component = get!(() -> Int[], components, find_root(sets, v))
    push!(component, v)
  end
  components
end

A graph with three components constructed as a coproduct of basic graphs:


In [ ]:
g = path_graph(Graph, 3) ⊕ cycle_graph(Graph, 4) ⊕ star_graph(Graph, 4)

values(connected_components(g))

:::{.rmenv title="Exercise (Julia macros)"}
Write a macro that creates a graph using a Graphviz-like syntax.
:::

:::{.solenv}
In Catlab, we typically use the pattern-matching features of [MLStyle.jl](https://github.com/thautwarm/MLStyle.jl) to simplify the parsing of Julia expressions. For the sake of illustration, we deconstruct the expressions "by hand" below.


In [ ]:
#| output: false
macro make_graph(expr)
  make_graph(expr)
end

function make_graph(block::Expr)
  g = Graph()
  vnames = Dict{Symbol,Int}()
  block.head == :block || error("Input to `make_graph` must be a block")
  for arg in block.args
    if arg isa LineNumberNode
      continue
    elseif arg isa Symbol
      haskey(vnames, arg) && error("Vertex $arg already defined")
      vnames[arg] = add_vertex!(g)
    elseif arg isa Expr && arg.head == :(->)
      sname, rhs = arg.args
      tname = rhs.args[2]
      add_edge!(g, vnames[sname], vnames[tname])
    else
      error("Cannot parse statement $arg")
    end
  end
  g
end

Let's try it out on the given example:


In [ ]:
g = @make_graph begin
  a
  b
  c
  a -> b
  b -> c
end

to_graphviz(g)

For comparison, Catlab ships with a similar macro `@graph` and a host of related but more complicated macros for building categories, diagrams in a category, and so on.
:::

## Part 3

:::{.rmenv title="Exercise (Lotka-Volterra with a twist, &starf; &starf;)"}
Suppose that there are populations of rocks, populations of papers, and populations of scissors. Predation happens to each based on the classic pattern; i.e. scissors eat papers, papers eat rocks, and rocks eat scissors.

Write down a model for a pairwise interaction between two implements based on Lotka-Volterra dynamics, and then compose three copies of that model to produce a three-population system.
:::

:::{.solenv}

First, let's define a resource sharing machine representing a predation process. This resource sharer has two state variables, one for the predator population and the other for the prey population. It is parameterized by a predation rate (i.e. how efficiently our predator inanimate objects eat the prey inanimate objects) and a growth rate (i.e. how efficiently our predators reproduce given how much they eat).


In [ ]:
using AlgebraicDynamics.UWDDynam
using Catlab
using Plots

# Return a continuous resource sharer describing the interaction
# of a predator and prey based on predation rate and growth rate parameters.
function binary_predation_system(growth_rate, predation_rate)
    dynamics(u,_,_) = [
        # The predators can reproduce if there are at least 2 of them.
        u[1] >= 2 ? growth_rate*u[1]*u[2] : 0,
        # The predators cannot eat if there aren't at least 1 of them.
        u[1] > 1 ? -predation_rate*u[1]*u[2] : 0
    ]
    return ContinuousResourceSharer{Float64}(2, dynamics)
end

Now, we can use Catlab to define the composition pattern of our rock-paper-scissors system.


In [ ]:
d = @relation (rocks, papers, scissors) begin
    crush(rocks, scissors)
    cover(papers, rocks)
    cut(scissors, papers)
end

to_graphviz(d, box_labels = :name, junction_labels = :variable, edge_attrs=Dict(:len => ".75"))

Now we can pick some rate parameters and fill our boxes with binary predation systems.


In [ ]:
crush_rate = .2
cover_rate = .1
cut_rate = .3

rock_reproduction_rate = 0.1
paper_reproduction_rate = 0.2
scissor_reproduction_rate = 0.1

subsystems = Dict(
    :crush => binary_predation_system(rock_reproduction_rate, crush_rate),
    :cover => binary_predation_system(paper_reproduction_rate, cover_rate),
    :cut => binary_predation_system(scissor_reproduction_rate, cut_rate)
)

rps_system = oapply(d, subsystems)

To see the results of our game, we can discretize the system using Euler's method and plot a trajectory.


In [ ]:
# Helper functions

# Run a DDS s from initial state x0 for iters number of iterations.
function simulate(s, x0, iters)
    trajectory = [x0]
    xk = x0
    for i in 1:iters
        xk = eval_dynamics(s, xk)
        push!(trajectory, xk)
    end
    return trajectory
end
# Get the trajectory of a specific population.
get_traj(trajs, i::Int) = [trajs[j][i] for j in 1:length(trajs)]

rps_discrete_system = euler_approx(rps_system, 0.01)

sol = simulate(rps_discrete_system, [100.0, 125.0, 90.0], 100)

rock_traj = get_traj(sol, 1)
paper_traj = get_traj(sol, 2)
scissor_traj = get_traj(sol, 3)

plt = plot(rock_traj, label="Rock")
plot!(paper_traj, label="Paper")
plot!(scissor_traj, label="Scissors")

display(plt)

As we can see, this choice of parameters has led us to a steady state position where rock has won!

:::